In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages TargetHolding:pyspark-cassandra:0.3.5,org.apache.spark:spark-streaming-kafka_2.10:1.6.1 pyspark-shell'


In [2]:
from pyspark import SparkContext
sc = SparkContext("local[2]", "KafkaReceiveStream") 


In [3]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pyspark_cassandra

In [4]:
ssc = StreamingContext(sc, 5)

In [5]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test': 1})
lines = kvs.map(lambda x: x[1])
content = lines.map(lambda line: line.split(',')).repartition(20)
content.pprint()
ssc.start()

-------------------------------------------
Time: 2016-05-15 10:13:15
-------------------------------------------



In [6]:
ssc.stop()